In [42]:
#packages
import numpy as np
import pandas as pd
import datetime

pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

In [43]:
#freeze header and allow for scroll through the table
from ipywidgets import interact, IntSlider
from IPython.display import display

def freeze_header(df, num_rows=30, step_rows=1):
    @interact(last_row=IntSlider(min=min(num_rows, df.shape[0]),
                                 max=df.shape[0],
                                 step=step_rows,
                                 description='rows',
                                 readout=False,
                                 disabled=False,
                                 continuous_update=True,
                                 orientation='horizontal',
                                 slider_color='purple'))
    def _freeze_header(last_row):
        display(df.iloc[max(0, last_row-num_rows):last_row,])

In [44]:
#read in data
shots = pd.read_csv("pbp_shots.csv")
pbp = pd.read_csv("pbp_full.csv")
gameflow_players = pd.read_csv("game_flow_players.csv")
gameflow_lineups = pd.read_csv("game_flow_lineups.csv")

gameflow_players_add = pd.read_csv("game_flow_players_sc6.csv")
gameflow_lineups_add = pd.read_csv("game_flow_lineups_sc6.csv")

In [47]:
gameflow_lineups = pd.concat([gameflow_lineups,gameflow_lineups_add])

In [49]:
gamelineups = gameflow_lineups['lineupId'].tolist()
gamenewlineups = []
for lineup in gamelineups:
    gamenewlineups.append(lineup.split('-'))

gameflow_lineups['newLineupId'] = gamenewlineups
gameflow_lineups['subs'] = gameflow_lineups['newLineupId'].map(set).diff().fillna(0)
gameflow_lineups['numSubs'] = gameflow_lineups['subs'].str.len().fillna(0)
gameflow_lineups['flowStint'] = gameflow_lineups['flowStint'].astype(int)
gameflow_lineups.loc[gameflow_lineups["flowStint"] == 1,"numSubs"] = 0.0
gameflow_lineups['numSubs'] = gameflow_lineups['numSubs'].astype(int)
gameflow_lineups['cStart'] = pd.to_datetime(gameflow_lineups['cStart'], format='%M:%S').dt.time

time_before = datetime.time(0, 5, 0)

def label_quarter (row):
   if row['cStart'] > time_before:
      return 'firstfive'
   return 'lastfive'

gameflow_lineups['quarterPart'] = gameflow_lineups.apply(lambda row: label_quarter(row), axis=1)

gameflow_lineups = gameflow_lineups[['rowId','flowStint','gameId','teamMarket','teamName','lineupId','periodNumber',
                                    'gameFlowPeriod','sStart','sEnd','cStart','cEnd','secs',
                                    'newLineupId','subs','numSubs','quarterPart']]

In [50]:
res = pd.DataFrame()
teamlist = []
for gameID in gameflow_lineups['gameId'].unique():
    game = gameflow_lineups[gameflow_lineups['gameId']==gameID].reset_index()
    gameinds = game[game['flowStint']==1].index
    team1 = game.iloc[gameinds[0]:gameinds[1]]
    team2 = game.iloc[gameinds[1]:]
    team1namefirst = team1.iloc[0]['teamMarket']
    team1namelast = team1.iloc[0]['teamName']
    team2namefirst = team2.iloc[0]['teamMarket']
    team2namelast = team2.iloc[0]['teamName']
    teamlist.append(team1namefirst+" "+team1namelast)
    teamlist.append(team2namefirst+" "+team2namelast)
    
    df = pd.DataFrame()
    df['game'] = team1.groupby(['periodNumber','quarterPart'])['teamMarket'].first()+" "+team1.groupby(['periodNumber','quarterPart'])['teamName'].first()+" vs. "+team2.groupby(['periodNumber','quarterPart'])['teamMarket'].first()+" "+team2.groupby(['periodNumber','quarterPart'])['teamName'].first()
    #df['team1Changes'] = team1.groupby(['periodNumber','quarterPart']).size()-1
    df['team1Subs'] = team1.groupby(['periodNumber','quarterPart'])['numSubs'].agg('sum')
    #df['team2Changes'] = team2.groupby(['periodNumber','quarterPart']).size()-1
    df['team2Subs'] = team2.groupby(['periodNumber','quarterPart'])['numSubs'].agg('sum')
    
    res = pd.concat([res,df])
    
res = res.reset_index()
res = res.fillna(0.0)
res = res.set_index(['game','periodNumber','quarterPart'])
res = res.astype(int)
res = res.reset_index()
res['game'] = res['game'].replace(to_replace=0, method='ffill')
res = res.set_index(['game','periodNumber','quarterPart'])

freeze_header(df=res, num_rows=16)

interactive(children=(IntSlider(value=16, description='rows', max=384, min=16, readout=False), Output()), _dom…

In [51]:
#number of games we have from each team
pd.Series(teamlist)[pd.Series(teamlist).isin(['UCLA Bruins','Virginia Tech Hokies','South Carolina Gamecocks','LSU Tigers','Iowa Hawkeyes'])].value_counts()

UCLA Bruins                 20
Iowa Hawkeyes                9
Virginia Tech Hokies         8
South Carolina Gamecocks     8
LSU Tigers                   7
dtype: int64

In [52]:
#res.to_csv("team_sub_patterns.csv")

In [56]:
res2 = pd.read_csv("team_subs_patterns_final.csv")
res2 = pd.DataFrame(res2.groupby(['game','periodNumber','quarterPart'])['teamSubs'].agg(sum))

In [57]:
newCol = []
for index, row in res2.iterrows():
    if index[0] == 'UCLA Bruins': newCol.append(row['teamSubs']/20)
    elif index[0] == 'Iowa Hawkeyes': newCol.append(row['teamSubs']/9)
    elif index[0] == 'Virginia Tech Hokies': newCol.append(row['teamSubs']/8)
    elif index[0] == 'LSU Tigers': newCol.append(row['teamSubs']/7)
    elif index[0] == 'South Carolina Gamecocks': newCol.append(row['teamSubs']/8)
res2['adjTeamSubs'] = np.round(newCol,2)

In [58]:
res2

teamSubs  adjTeamSubs
game                     periodNumber quarterPart                       
Iowa Hawkeyes            1            firstfive          14         1.56
                                      lastfive           57         6.33
                         2            firstfive          24         2.67
                                      lastfive           56         6.22
                         3            firstfive          22         2.44
                                      lastfive           40         4.44
                         4            firstfive          22         2.44
                                      lastfive           26         2.89
LSU Tigers               1            firstfive           7         1.00
                                      lastfive           28         4.00
                         2            firstfive          29         4.14
                                      lastfive           22         3.14
                         3            firstfive          25         3.57
                                      lastfive           22         3.14
                         4            firstfive          16         2.29
                                      lastfive           20         2.86
South Carolina Gamecocks 1            firstfive           9         1.12
                                      lastfive           37         4.62
                         2            firstfive          31         3.88
                                      lastfive           20         2.50
                         3            firstfive          28         3.50
                                      lastfive           18         2.25
                         4            firstfive          16         2.00
                                      lastfive           35         4.38
UCLA Bruins              1            firstfive          71         3.55
                                      lastfive          150         7.50
                         2            firstfive         117         5.85
                                      lastfive           96         4.80
                         3            firstfive         116         5.80
                                      lastfive          100         5.00
                         4            firstfive         104         5.20
                                      lastfive          195         9.75
Virginia Tech Hokies     1            firstfive          10         1.25
                                      lastfive           30         3.75
                         2            firstfive          21         2.62
                                      lastfive           13         1.62
                         3            firstfive          15         1.88
                                      lastfive           14         1.75
                         4            firstfive          15         1.88
                                      lastfive           24         3.00

In [59]:
#res2.to_csv("team_sub_patterns_agg.csv")